# Dual-Pipeline Trainer: 23 Vision+Tabular Experiments

This workbook orchestrates a two-lane workflow:
- **Listings that include imagery** rely on 23 different hybrid CNN + tabular blueprints.
- **Rows without pictures** are routed to a purpose-built dense network that only consumes spreadsheet fields.

### Why this version?
- ▶ Uses the current Keras `.keras` serialization so weights stay portable.
- ▶ Heavy feature engineering plus quantile scaling keep the tabular branch stable.
- ▶ Cosine LR schedules, mixup, and mixed precision squeeze more signal out of limited data.
- ▶ A lightweight ensemble stage can blend the strongest checkpoints.
- ▶ Loss functions, callbacks, and logging were tuned specifically for property pricing.

### Vision backbones on duty
AlexNet, NIN, ZFNet, VGG16/VGG19, GoogLeNet, InceptionV3/V4, Highway, ResNet50/101/152, InceptionResNetV2, FractalNet, WideResNet, Xception, ResidualAttention, SENet, DenseNet121/201, MobileNetV2, CapsuleNet, HRNet.

### Tabular specialist
A deep fully-connected model with residual skips, attention-style gates, and auxiliary supervision handles the image-free properties.

### Artifacts
Checkpoints, preprocessors, and reports are written to `D:\houses_saved` whenever saving is enabled.

In [3]:
import os

# Ensure the global save toggle exists even if the configuration cell hasn't run yet
ENABLE_MODEL_SAVING = globals().get('ENABLE_MODEL_SAVING', True)

# Suppress verbose CUDA init errors on systems without supported GPUs
os.environ.setdefault('TF_CPP_MIN_LOG_LEVEL', '3')  # Silence INFO/WARN logs from CUDA
os.environ.setdefault('CUDA_VISIBLE_DEVICES', '-1')

import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, callbacks, Input, mixed_precision

# Silence TensorFlow logger to avoid noisy cuInit messages
import logging
log = tf.get_logger()
log.setLevel('ERROR')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, ResNet101, ResNet152, 
    InceptionV3, InceptionResNetV2, Xception, 
    DenseNet121, DenseNet201, MobileNetV2
)
from tensorflow.keras.applications import (
    vgg16, vgg19, resnet, inception_v3, inception_resnet_v2, 
    xception, densenet, mobilenet_v2
)
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from datetime import datetime
import joblib
import json
import gc
import warnings
warnings.filterwarnings('ignore')

# Suppress HDF5 format warnings specifically
logging.getLogger('absl').setLevel(logging.ERROR)

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import RobustScaler, OneHotEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

# Enable mixed precision for better performance
mixed_precision.set_global_policy('mixed_float16')
HAS_GPU = len(tf.config.list_physical_devices('GPU')) > 0
if not HAS_GPU:
    # Fall back to float32 on CPU-only hosts to avoid AVX-512 / oneDNN warnings
    mixed_precision.set_global_policy('float32')
    try:
        tf.config.set_visible_devices([], 'GPU')
    except Exception:
        pass

# Force Keras format for all model saving operations
import os
os.environ['TF_KERAS_SAVE_FORMAT'] = 'keras'

# Set TensorFlow to use Keras format by default
tf.keras.utils.set_random_seed(42)  # For reproducibility

# Global placeholders for target standardization (populated after data load)
LOG_PRICE_MEAN = None
LOG_PRICE_STD = None

# OPTIMIZED Configuration - Enhanced for better performance with hyperparameter tuning
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  # Balanced mini-batch for stable gradients
EPOCHS = 60      # Cap epochs to prevent over-training and save time
FINE_TUNE_EPOCHS = 18  # Shorter refinement stage to match new schedule
INITIAL_LR = 5e-5  # Lower initial LR for more stable training (reduced from 1.5e-4)
FINE_TUNE_LR = 1e-6  # Very low fine-tune LR for gradual adaptation
MIN_LR = 5e-6  # Prevent LR from collapsing and keep progress steady
TARGET_VAL_MAE = 0.5  # User-specified requirement (in log space)
EARLY_STOP_MAE = TARGET_VAL_MAE
IMAGE_DIR = Path('images')
SAVE_DIR = Path(r"D:\houses_saved")

# OPTIMIZED Advanced training parameters - Tuned for better performance
USE_AUGMENTATION = True
USE_MIXUP = True
MIXUP_ALPHA = 0.2  # Slightly increased for better regularization
LABEL_SMOOTHING = 0.1  # Increased for better generalization
DROPOUT_RATE = 0.35  # Increased dropout for better regularization
L2_REG = 1e-4  # Increased L2 regularization for better generalization
GRADIENT_CLIP_NORM = 1.0  # Gradient clipping for stability
WARMUP_EPOCHS = 5  # More warmup epochs for stable training

# Hyperparameter tuning ranges
LR_SEARCH_RANGE = [1e-5, 5e-5, 1e-4]  # Learning rate search space
DROPOUT_SEARCH_RANGE = [0.3, 0.35, 0.4]  # Dropout search space
BATCH_SIZE_SEARCH = [24, 32, 40]  # Batch size search space

if ENABLE_MODEL_SAVING:
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"[Init] Models will be written to {SAVE_DIR}")
else:
    print("↪ Model saving disabled: training will not write new artifacts.")
print(f"[Init] TensorFlow {tf.__version__} | GPU detected: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"[Init] Mixed precision policy: {mixed_precision.global_policy().name}")

# Configure GPU memory growth if available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"[Init] GPU memory growth configured for {len(gpus)} device(s)")
    except RuntimeError as e:
        print(f"⚠ GPU configuration warning: {e}")

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# Toggle whether training should write any new artifacts to disk
ENABLE_MODEL_SAVING = True  # Set to True if you want to generate fresh checkpoints/reports


def safe_artifact_write(description, path, save_fn):
    """Write to disk only when saving is enabled; otherwise show a skip notice."""
    path = Path(path)
    if not ENABLE_MODEL_SAVING:
        print(f"↪ Skipping save for {description} (saving disabled).")
        return False
    try:
        save_fn()
        print(f"[Save] {description} -> {path}")
        return True
    except Exception as err:
        print(f"⚠ Failed to save {description}: {err}")
        return False


class SkipCheckpointNotice(callbacks.Callback):
    """Placeholder callback that just informs the user that saving is disabled."""
    def __init__(self, filepath):
        super().__init__()
        self.filepath = Path(filepath)

    def on_train_begin(self, logs=None):
        print(f"↪ Model checkpoints disabled. Training will continue without writing {self.filepath.name}.")



BEST_LEDGER_PATH = SAVE_DIR / "best_r2_ledger.json"

def _load_best_model_ledger():
    if not BEST_LEDGER_PATH.exists():
        return {}
    try:
        with open(BEST_LEDGER_PATH, 'r') as fh:
            return json.load(fh)
    except Exception as ledger_error:
        print(f"??? Best-model ledger unreadable: {ledger_error}")
        return {}

BEST_MODEL_LEDGER = _load_best_model_ledger()

def persist_best_model_ledger():
    if not ENABLE_MODEL_SAVING:
        return False
    def _write(path=BEST_LEDGER_PATH):
        with open(path, 'w') as fh:
            json.dump(BEST_MODEL_LEDGER, fh, indent=2)
    return safe_artifact_write("R^2 ledger", BEST_LEDGER_PATH, _write)

def should_update_best_model(model_key, candidate_r2):
    best = BEST_MODEL_LEDGER.get(model_key)
    if best is None:
        return True
    return candidate_r2 > best.get('r2', float('-inf'))


NameError: name 'callbacks' is not defined

## 1. Data Refinement, Feature Crafting, and Image Augmentations

In [3]:
def create_advanced_features(df):
    """OPTIMIZED: Create advanced engineered features with more sophisticated transformations"""
    # Price per square meter features
    df['price_per_building_area'] = df['price(USD)'] / (df['building_area(m²)'] + 1)
    df['price_per_land_area'] = df['price(USD)'] / (df['land_area(m²)'] + 1)
    
    # Area ratios and interactions
    df['area_ratio'] = df['building_area(m²)'] / (df['land_area(m²)'] + 1)
    df['total_area'] = df['building_area(m²)'] + df['land_area(m²)']
    df['area_difference'] = df['land_area(m²)'] - df['building_area(m²)']
    df['area_utilization'] = df['building_area(m²)'] / (df['total_area'] + 1)  # NEW: Utilization ratio
    
    # Bedroom density and efficiency
    df['bedrooms_per_area'] = df['bedrooms'] / (df['building_area(m²)'] + 1)
    df['area_per_bedroom'] = df['building_area(m²)'] / (df['bedrooms'] + 1)
    df['bedroom_density'] = df['bedrooms'] / (df['total_area'] + 1)  # NEW: Overall density
    
    # NEW: Price efficiency metrics
    df['price_efficiency'] = df['building_area(m²)'] / (df['price(USD)'] + 1)
    df['value_score'] = (df['building_area(m²)'] * df['bedrooms']) / (df['price(USD)'] + 1)
    
    # Location-based statistical features (OPTIMIZED)
    if 'location' in df.columns:
        location_stats = df.groupby('location')['price(USD)'].agg(['mean', 'median', 'std', 'count', 'min', 'max']).fillna(0)
        location_stats.columns = [f'location_{col}' for col in location_stats.columns]
        df = df.merge(location_stats, left_on='location', right_index=True)
        
        # Location price percentile
        df['location_price_percentile'] = df.groupby('location')['price(USD)'].rank(pct=True)
        
        # NEW: Location-based area statistics
        location_area_stats = df.groupby('location')['building_area(m²)'].agg(['mean', 'std']).fillna(0)
        location_area_stats.columns = [f'location_area_{col}' for col in location_area_stats.columns]
        df = df.merge(location_area_stats, left_on='location', right_index=True)
    
    # Log transformations for skewed features
    for col in ['building_area(m²)', 'land_area(m²)', 'total_area', 'price(USD)']:
        if col in df.columns and (df[col] > 0).any():
            df[f'log_{col}'] = np.log1p(df[col])
    
    # OPTIMIZED: Polynomial and interaction features
    df['bedrooms_squared'] = df['bedrooms'] ** 2
    df['area_ratio_squared'] = df['area_ratio'] ** 2
    df['bedrooms_area_interaction'] = df['bedrooms'] * df['building_area(m²)']  # NEW: Interaction
    df['price_area_interaction'] = df['price(USD)'] * df['building_area(m²)'] / 1000  # NEW: Scaled interaction
    
    # NEW: Binned features for non-linear relationships
    df['bedrooms_binned'] = pd.cut(df['bedrooms'], bins=[0, 1, 2, 3, 5, 10], labels=[1, 2, 3, 4, 5]).astype(float)
    df['area_binned'] = pd.qcut(df['building_area(m²)'], q=5, labels=False, duplicates='drop').astype(float)
    
    return df

def create_image_augmentation():
    """Create advanced image augmentation pipeline"""
    return tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
        layers.RandomContrast(0.1),
        layers.RandomTranslation(0.1, 0.1),
    ])

def mixup_data(x, y, alpha=0.2):
    """Implement mixup data augmentation"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = tf.shape(x)[0]
    index = tf.random.shuffle(tf.range(batch_size))
    
    mixed_x = lam * x + (1 - lam) * tf.gather(x, index)
    mixed_y = lam * y + (1 - lam) * tf.gather(y, index)
    
    return mixed_x, mixed_y

import contextlib
import sys
from io import StringIO

@contextlib.contextmanager
def suppress_hdf5_warnings():
    """Context manager to suppress HDF5 format warnings"""
    # Capture stderr to suppress warnings
    old_stderr = sys.stderr
    sys.stderr = StringIO()
    
    # Also suppress absl warnings
    old_level = logging.getLogger('absl').level
    logging.getLogger('absl').setLevel(logging.CRITICAL)
    
    try:
        yield
    finally:
        sys.stderr = old_stderr
        logging.getLogger('absl').setLevel(old_level)

class KerasModelCheckpoint(callbacks.ModelCheckpoint):
    """Custom ModelCheckpoint that forces Keras format and suppresses warnings"""
    def __init__(self, filepath, *args, **kwargs):
        filepath = str(filepath)
        if not filepath.endswith('.keras'):
            filepath = f"{filepath}.keras"
        kwargs.pop('save_format', None)  # Deprecated in modern Keras
        kwargs.setdefault('save_weights_only', False)
        super().__init__(filepath, *args, **kwargs)
    
    def _save_model(self, epoch, batch, logs):
        """Override to ensure Keras format and suppress warnings"""
        if not ENABLE_MODEL_SAVING:
            print(f"↪ Skipping checkpoint save for {Path(self.filepath).name} (saving disabled).")
            return
        with suppress_hdf5_warnings():
            try:
                super()._save_model(epoch, batch, logs)
            except Exception as e:
                if hasattr(self, 'model') and self.model is not None:
                    try:
                        self.model.save(self.filepath, overwrite=True)
                    except Exception as save_error:
                        print(f"Warning: Model save failed: {save_error}")


class GeneralizationGuard(callbacks.Callback):
    """Dynamically lower the learning rate if validation drifts away from training."""
    def __init__(self, monitor='val_main_output_mae', reference='main_output_mae',
                 gap=0.04, patience=3, decay=0.5, start_epoch=12, min_lr=MIN_LR,
                 max_adjustments=4):
        super().__init__()
        self.monitor = monitor
        self.reference = reference
        self.gap = gap
        self.patience = patience
        self.decay = decay
        self.start_epoch = start_epoch
        self.min_lr = min_lr
        self.max_adjustments = max_adjustments
        self.bad_epochs = 0
        self.adjustments = 0
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if epoch < self.start_epoch or self.adjustments >= self.max_adjustments:
            return
        train_score = logs.get(self.reference)
        val_score = logs.get(self.monitor)
        if train_score is None or val_score is None:
            return
        if (val_score - train_score) > self.gap:
            self.bad_epochs += 1
        else:
            self.bad_epochs = 0
        if self.bad_epochs >= self.patience:
            current_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
            new_lr = max(current_lr * self.decay, self.min_lr)
            # Be robust to different optimizer APIs (lr vs learning_rate, schedules, etc.)
            try:
                lr_var = getattr(self.model.optimizer, 'learning_rate', None)
                if lr_var is None:
                    lr_var = getattr(self.model.optimizer, 'lr', None)
                if hasattr(lr_var, 'assign'):
                    lr_var.assign(new_lr)
                else:
                    tf.keras.backend.set_value(lr_var, new_lr)
            except Exception:
                # If we cannot adjust lr safely, just log and continue without crashing
                print("  ↪ GeneralizationGuard could not modify optimizer LR; skipping LR update for this epoch.")
                return
            self.adjustments += 1
            self.bad_epochs = 0
            print(f"  ↪ GeneralizationGuard reduced LR from {current_lr:.2e} to {new_lr:.2e} (train/val gap detected).")

def freeze_entire_backbone(backbone):
    """Freeze all backbone layers (except keep BN in inference mode)."""
    if backbone is None:
        return
    for layer in backbone.layers:
        layer.trainable = False
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False

def unfreeze_backbone_fraction(backbone, fraction=0.4):
    """Unfreeze the top fraction of layers for fine-tuning."""
    if backbone is None:
        return
    total_layers = len(backbone.layers)
    start_idx = int(total_layers * (1 - fraction))
    for idx, layer in enumerate(backbone.layers):
        layer.trainable = idx >= start_idx


MULTIMODAL_LOSSES = {
    'main_output': tf.keras.losses.Huber(delta=1.0),
    'aux_output': tf.keras.losses.Huber(delta=1.0)
}
MULTIMODAL_METRICS = {
    'main_output': ['mae', 'mse'],
    'aux_output': ['mae']
}


def compile_multimodal(model, learning_rate):
    """Utility to (re)compile multimodal models with shared settings."""
    model.compile(
        optimizer=optimizers.AdamW(learning_rate=learning_rate, weight_decay=L2_REG, clipnorm=1.0),
        loss=MULTIMODAL_LOSSES,
        loss_weights={'main_output': 1.0, 'aux_output': 0.3},
        metrics=MULTIMODAL_METRICS
    )

def build_multimodal_callbacks(filepath, patience=15, use_scheduler=True, lr_epochs=EPOCHS, initial_lr=INITIAL_LR):
    """Shared callback builder for baseline + fine-tune stages with enhanced monitoring."""
    callback_list = []
    if ENABLE_MODEL_SAVING:
        callback_list.append(
            KerasModelCheckpoint(
                filepath,
                save_best_only=True,
                monitor='val_main_output_mae',
                mode='min',
                verbose=1,
                save_freq='epoch'
            )
        )
    else:
        callback_list.append(SkipCheckpointNotice(filepath))

    callback_list.append(
        callbacks.ReduceLROnPlateau(
            monitor='val_main_output_mae',
            factor=0.5,  # More aggressive reduction
            patience=max(5, patience // 2),  # Reduced patience for faster adaptation
            min_lr=MIN_LR,
            mode='min',
            verbose=1,
            cooldown=2
        )
    )
    if use_scheduler:
        # Cosine annealing with warm restarts for better exploration
        def cosine_annealing_with_restarts(epoch):
            restart_period = max(1, lr_epochs // 3)
            cycle = epoch // restart_period
            epoch_in_cycle = epoch % restart_period
            lr = initial_lr * 0.5 * (1 + np.cos(np.pi * epoch_in_cycle / restart_period))
            return max(lr, MIN_LR)
        
        callback_list.append(
            callbacks.LearningRateScheduler(
                cosine_annealing_with_restarts,
                verbose=0
            )
        )
    callback_list.append(
        GeneralizationGuard(
            monitor='val_main_output_mae',
            reference='main_output_mae',
            gap=0.035,
            patience=3,
            start_epoch=15,
            min_lr=MIN_LR,
            decay=0.6
        )
    )
    callback_list.append(callbacks.TerminateOnNaN())
    if ENABLE_MODEL_SAVING:
        callback_list.append(
            callbacks.CSVLogger(str(SAVE_DIR / 'training_log.csv'), append=True)
        )
    return callback_list


def evaluate_actual_metrics(log_preds, log_targets):
    """Return MAE, MSE and R² in the original price space."""
    global LOG_PRICE_MEAN, LOG_PRICE_STD
    if LOG_PRICE_MEAN is None or LOG_PRICE_STD is None:
        raise ValueError("LOG_PRICE_MEAN/STD must be set before evaluating metrics.")
    # De-standardize back to log-price space before exponentiating
    log_preds_actual = (log_preds * LOG_PRICE_STD) + LOG_PRICE_MEAN
    log_targets_actual = (log_targets * LOG_PRICE_STD) + LOG_PRICE_MEAN
    preds_actual = np.expm1(log_preds_actual)
    actual = np.expm1(log_targets_actual)
    actual_mae = mean_absolute_error(actual, preds_actual)
    actual_mse = mean_squared_error(actual, preds_actual)
    actual_r2 = r2_score(actual, preds_actual)
    return actual_mae, actual_mse, actual_r2


def make_price_strata(prices, max_bins=5):
    """Create stratification labels that preserve the price distribution."""
    prices = prices.fillna(prices.median())
    unique_vals = prices.nunique()
    bins = min(max_bins, max(2, unique_vals))
    try:
        labels = pd.qcut(prices, q=bins, labels=False, duplicates='drop')
    except ValueError:
        labels = pd.Series(0, index=prices.index)
    return labels.astype(int)


def calibrate_log_predictions(train_preds_std, train_targets_std, val_preds_std):
    """Fit a linear calibration on train predictions and apply it to validation predictions."""
    train_preds_std = np.asarray(train_preds_std).reshape(-1, 1)
    train_targets_std = np.asarray(train_targets_std).reshape(-1, 1)
    val_preds_std = np.asarray(val_preds_std).reshape(-1, 1)
    reg = LinearRegression()
    reg.fit(train_preds_std, train_targets_std)
    calibrated_val = reg.predict(val_preds_std).reshape(-1)
    slope = float(reg.coef_[0][0] if reg.coef_.ndim > 1 else reg.coef_[0])
    intercept = float(reg.intercept_[0] if np.ndim(reg.intercept_) > 0 else reg.intercept_)
    return calibrated_val, {'slope': slope, 'intercept': intercept}


def evaluate_with_calibration(train_preds_std, train_targets_std, val_preds_std, val_targets_std):
    """Compare baseline vs calibrated metrics in actual price space."""
    baseline_metrics = evaluate_actual_metrics(val_preds_std, val_targets_std)
    if train_preds_std is not None and len(train_preds_std) > 0:
        calibrated_val_std, cal_params = calibrate_log_predictions(train_preds_std, train_targets_std, val_preds_std)
        calibrated_metrics = evaluate_actual_metrics(calibrated_val_std, val_targets_std)
    else:
        calibrated_val_std = val_preds_std
        calibrated_metrics = baseline_metrics
        cal_params = {'slope': 1.0, 'intercept': 0.0}
    return baseline_metrics, calibrated_metrics, cal_params, calibrated_val_std

def find_optimal_learning_rate(model, X_train, y_train, X_val, y_val, min_lr=1e-7, max_lr=1e-2, steps=50):
    """Find optimal learning rate using learning rate range test."""
    import math
    
    lrs = np.logspace(np.log10(min_lr), np.log10(max_lr), steps)
    losses = []
    
    for lr in lrs:
        model.compile(
            optimizer=optimizers.AdamW(learning_rate=lr, weight_decay=L2_REG, clipnorm=GRADIENT_CLIP_NORM),
            loss=MULTIMODAL_LOSSES,
            loss_weights={'main_output': 1.0, 'aux_output': 0.3},
            metrics=MULTIMODAL_METRICS
        )
        # Train for a few steps
        hist = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=0
        )
        losses.append(hist.history['val_loss'][0])
    
    # Find the learning rate with steepest descent
    best_idx = np.argmin(losses)
    optimal_lr = lrs[best_idx]
    
    # Use a value slightly lower than optimal for stability
    optimal_lr = optimal_lr * 0.5
    
    return optimal_lr

def hyperparameter_search(model_fn, X_train, y_train, X_val, y_val, param_grid, max_combinations=5):
    """Perform grid search for hyperparameter tuning."""
    best_score = float('inf')
    best_params = None
    results = []
    
    # Limit combinations to avoid too many trials
    combinations = []
    for lr in param_grid.get('learning_rate', [INITIAL_LR]):
        for dropout in param_grid.get('dropout', [DROPOUT_RATE]):
            for batch_size in param_grid.get('batch_size', [BATCH_SIZE]):
                combinations.append({'lr': lr, 'dropout': dropout, 'batch_size': batch_size})
    
    # Limit to max_combinations
    combinations = combinations[:max_combinations]
    
    print(f"  🔍 Hyperparameter search: Testing {len(combinations)} combinations...")
    
    for i, params in enumerate(combinations, 1):
        print(f"  [{i}/{len(combinations)}] Testing LR={params['lr']:.2e}, Dropout={params['dropout']:.2f}, Batch={params['batch_size']}")
        
        try:
            # Create model with current params
            model = model_fn(dropout_rate=params['dropout'])
            
            # Compile with current learning rate
            model.compile(
                optimizer=optimizers.AdamW(learning_rate=params['lr'], weight_decay=L2_REG, clipnorm=GRADIENT_CLIP_NORM),
                loss=MULTIMODAL_LOSSES,
                loss_weights={'main_output': 1.0, 'aux_output': 0.3},
                metrics=MULTIMODAL_METRICS
            )
            
            # Quick training for evaluation (few epochs)
            # X_train and y_train are already in the correct format for multimodal
            hist = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=5,  # Quick evaluation - reduced from 10
                batch_size=params['batch_size'],
                verbose=0
            )
            
            best_val_mae = min(hist.history['val_main_output_mae'])
            results.append({
                'params': params,
                'val_mae': best_val_mae
            })
            
            if best_val_mae < best_score:
                best_score = best_val_mae
                best_params = params
            
            print(f"      → Val MAE: {best_val_mae:.4f}")
            
            del model
            tf.keras.backend.clear_session()
            gc.collect()
            
        except Exception as e:
            print(f"      ✗ Failed: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    if best_params is None:
        # Fallback to defaults if all failed
        best_params = {'lr': INITIAL_LR, 'dropout': DROPOUT_RATE, 'batch_size': BATCH_SIZE}
        best_score = float('inf')
        print(f"  ⚠ All hyperparameter searches failed, using defaults")
    else:
        print(f"      Candidate summary -> LR={best_params['lr']:.2e}, Dropout={best_params['dropout']:.2f}, Batch={best_params['batch_size']} (MAE {best_score:.4f})")
    return best_params, results


print("[Setup] Feature engineering helpers ready")


[Setup] Feature engineering helpers ready


In [4]:
def remove_outliers_iqr(df, column):
    Q1, Q3 = df[column].quantile(0.25), df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] >= Q1 - 1.5*IQR) & (df[column] <= Q3 + 1.5*IQR)]

def load_and_clean_data(csv_path='property_final.csv'):
    df = pd.read_csv(csv_path)
    print(f"[Ingest] Columns detected -> {list(df.columns)}")
    
    df['price(USD)'] = pd.to_numeric(df['price(USD)'], errors='coerce')
    df = df.dropna(subset=['price(USD)'])
    
    # NOTE: property_final.csv does NOT have a 'bathrooms' column
    for col in ['building_area(m²)', 'land_area(m²)', 'bedrooms']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    df = df[(df['price(USD)'] > 5000) & (df['price(USD)'] < 3000000)]
    df = df[df['bedrooms'] < 10]
    
    mask_building = (df['building_area(m²)'] > 0)
    if mask_building.any():
        Q1 = df.loc[mask_building, 'building_area(m²)'].quantile(0.25)
        Q3 = df.loc[mask_building, 'building_area(m²)'].quantile(0.75)
        upper = Q3 + 2.0 * (Q3 - Q1)
        df = df[~mask_building | (df['building_area(m²)'] <= upper)]
    
    df = remove_outliers_iqr(df, 'price(USD)')
    
    # Apply advanced feature engineering
    df = create_advanced_features(df)
    
    # Target transformations
    df['log_price'] = np.log1p(df['price(USD)'])
    df['sqrt_price'] = np.sqrt(df['price(USD)'])
    
    # Standardize log_price target (mean/std across entire dataset)
    global LOG_PRICE_MEAN, LOG_PRICE_STD
    LOG_PRICE_MEAN = df['log_price'].mean()
    LOG_PRICE_STD = df['log_price'].std() if df['log_price'].std() != 0 else 1.0
    df['log_price_std'] = (df['log_price'] - LOG_PRICE_MEAN) / LOG_PRICE_STD
    
    print(f"[Ingest] Cleaned frame -> shape {df.shape}, log_price μ={LOG_PRICE_MEAN:.4f}, σ={LOG_PRICE_STD:.4f}")
    return df

df = load_and_clean_data().reset_index(drop=True)

def get_image_path(row_id):
    """Get image path. The id column already contains the full filename."""
    if pd.isna(row_id) or row_id == '':
        return None
    
    # Try the id as-is (since it already contains the extension)
    path = IMAGE_DIR / str(row_id)
    if path.exists():
        return str(path)
    
    # If not found, try stripping extension and adding common ones
    base_name = str(row_id).rsplit('.', 1)[0]
    for ext in ['.jpg', '.jpeg', '.png', '.webp']:
        path = IMAGE_DIR / f"{base_name}{ext}"
        if path.exists():
            return str(path)
    
    return None

def load_raw_image(path, size=IMG_SIZE):
    if path is None: return np.zeros((*size, 3), dtype=np.uint8)
    img = cv2.imread(path)
    if img is None: return np.zeros((*size, 3), dtype=np.uint8)
    return cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), size)

print("\n[Data] Resolving image references...")
df['image_path'] = df['id'].apply(get_image_path)
df_with_images = df[df['image_path'].notna()].copy()
df_no_images = df[df['image_path'].isna()].copy()
price_bins_with_images = make_price_strata(df_with_images['price(USD)']) if len(df_with_images) > 0 else None
price_bins_no_images = make_price_strata(df_no_images['price(USD)']) if len(df_no_images) > 0 else None

print(f"[Data] Records={len(df)} | image+tab={len(df_with_images)} | tab-only={len(df_no_images)}")

if len(df_with_images) > 0:
    print("   Preview image files:")
    for path in df_with_images['image_path'].head(3):
        print(f"      • {path}")

# Enhanced tabular preprocessing with feature selection
NUMERIC_FEATURES = [
    'building_area(m²)', 'land_area(m²)', 'bedrooms', 'area_ratio',
    'price_per_building_area', 'price_per_land_area', 'total_area', 
    'area_difference', 'bedrooms_per_area', 'area_per_bedroom',
    'bedrooms_squared', 'area_ratio_squared'
]

# Add log features if they exist
log_features = [col for col in df.columns if col.startswith('log_') and col != 'log_price']
NUMERIC_FEATURES.extend(log_features)

# Add location statistical features if they exist
location_features = [col for col in df.columns if col.startswith('location_')]
NUMERIC_FEATURES.extend(location_features)

CATEGORICAL_FEATURES = ['location'] if 'location' in df.columns else []

# Enhanced preprocessing pipeline
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', QuantileTransformer(output_distribution='normal')),  # Better for non-normal distributions
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, NUMERIC_FEATURES),
    ('cat', categorical_pipeline, CATEGORICAL_FEATURES)
])

# Fit and transform
X_all_tabular = preprocessor.fit_transform(df)

# Feature selection to reduce overfitting
if X_all_tabular.shape[1] > 50:  # Only if we have many features
    selector = SelectKBest(score_func=f_regression, k=min(50, X_all_tabular.shape[1]))
    X_all_tabular = selector.fit_transform(X_all_tabular, df['log_price'])
    selector_path = SAVE_DIR / 'feature_selector.joblib'
    safe_artifact_write(
        "Feature selector",
        selector_path,
        lambda path=selector_path: joblib.dump(selector, path)
    )
    print(f"[Features] Selector kept {X_all_tabular.shape[1]} signals")

# Save preprocessor
preprocessor_path = SAVE_DIR / 'preprocessor.joblib'
safe_artifact_write(
    "Preprocessor",
    preprocessor_path,
    lambda path=preprocessor_path: joblib.dump(preprocessor, path)
)
print(f"[Features] Matrix shape: {X_all_tabular.shape}")
print(f"[Features] Raw feature count: {len(NUMERIC_FEATURES + CATEGORICAL_FEATURES)}")

[Ingest] Columns detected -> ['id', 'title', 'price(USD)', 'building_area(m²)', 'land_area(m²)', 'bedrooms', 'location']
[Ingest] Cleaned frame -> shape (1102, 40), log_price μ=11.9134, σ=0.9621

[Data] Resolving image references...
[Data] Records=1102 | image+tab=256 | tab-only=846
   Preview image files:
      • images/df03d95d0b_0.webp
      • images/9255ebae71_0.webp
      • images/0aa8829830_0.webp
[Save] Feature selector -> D:\houses_saved/feature_selector.joblib
[Features] Selector kept 50 signals
[Save] Preprocessor -> D:\houses_saved/preprocessor.joblib
[Features] Matrix shape: (1102, 50)
[Features] Raw feature count: 27


In [5]:
# Global store for per-model performance summaries
model_performance = []



## 2. Backbone Library and Regularization Toolkit

In [6]:
# Enhanced architectures with better regularization and modern techniques

def build_alexnet(s):
    i = Input(s)
    x = layers.Conv2D(96, (11,11), strides=4, activation='relu', padding='valid', 
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(i)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    
    x = layers.Conv2D(256, (5,5), activation='relu', padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    
    x = layers.Conv2D(384, (3,3), activation='relu', padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(384, (3,3), activation='relu', padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same',
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    return Model(i, x)

def build_nin(s):
    i = Input(s)
    def mlpconv(x, filters):
        x = layers.Conv2D(filters, (5,5), padding='same', activation='relu')(x)
        x = layers.Conv2D(filters, (1,1), activation='relu')(x)
        x = layers.Conv2D(filters, (1,1), activation='relu')(x)
        return x
    x = mlpconv(i, 192)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = mlpconv(x, 192)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = mlpconv(x, 192)
    return Model(i, x)

def build_zfnet(s):
    i = Input(s)
    x = layers.Conv2D(96, (7,7), strides=2, activation='relu', padding='same')(i)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = layers.Conv2D(256, (5,5), strides=2, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    x = layers.Conv2D(384, (3,3), activation='relu', padding='same')(x)
    x = layers.Conv2D(384, (3,3), activation='relu', padding='same')(x)
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3,3), strides=2)(x)
    return Model(i, x)

def build_googlenet(s):
    i = Input(s)
    x = layers.Conv2D(64, (7,7), strides=2, padding='same', activation='relu')(i)
    x = layers.MaxPooling2D((3,3), strides=2, padding='same')(x)
    def inception_module(x, f1, f3r, f3, f5r, f5, proj):
        path1 = layers.Conv2D(f1, (1,1), padding='same', activation='relu')(x)
        path2 = layers.Conv2D(f3r, (1,1), padding='same', activation='relu')(x)
        path2 = layers.Conv2D(f3, (3,3), padding='same', activation='relu')(path2)
        path3 = layers.Conv2D(f5r, (1,1), padding='same', activation='relu')(x)
        path3 = layers.Conv2D(f5, (5,5), padding='same', activation='relu')(path3)
        path4 = layers.MaxPooling2D((3,3), strides=1, padding='same')(x)
        path4 = layers.Conv2D(proj, (1,1), padding='same', activation='relu')(path4)
        return layers.concatenate([path1, path2, path3, path4], axis=3)
    x = inception_module(x, 64, 96, 128, 16, 32, 32)
    x = layers.MaxPooling2D((3,3), strides=2, padding='same')(x)
    return Model(i, x)

def build_highway(s):
    i = Input(s)
    x = layers.Conv2D(64, (3,3), padding='same')(i)
    for _ in range(5):
        H = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
        T = layers.Conv2D(64, (3,3), padding='same', activation='sigmoid')(x)
        x = layers.Add()([layers.Multiply()([H, T]), layers.Multiply()([x, layers.Lambda(lambda z: 1.0-z)(T)])])
    return Model(i, x)

def build_inception_v4(s): 
    return InceptionV3(include_top=False, weights=None, input_shape=s)

def build_wideresnet(s):
    i = Input(s)
    x = layers.Conv2D(16, (3,3), padding='same', activation='relu')(i)
    k = 4
    x = layers.Conv2D(16*k, (3,3), padding='same', activation='relu')(x)
    x = layers.Conv2D(32*k, (3,3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(64*k, (3,3), strides=2, padding='same', activation='relu')(x)
    return Model(i, x)

def build_fractalnet(s):
    i = Input(s)
    c1 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(i)
    c2 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(c1)
    c3 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(c2)
    join = layers.Add()([c1, c3])
    x = layers.MaxPooling2D()(join)
    return Model(i, x)

def build_residual_attention(s):
    i = Input(s)
    x = layers.Conv2D(64, (7,7), strides=2, padding='same', activation='relu')(i)
    trunk = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    mask = layers.Conv2D(64, (3,3), padding='same', activation='sigmoid')(x)
    out = layers.Multiply()([trunk, mask])
    out = layers.Add()([out, trunk])
    return Model(i, out)

def build_senet(s):
    i = Input(s)
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu',
                     kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(i)
    x = layers.BatchNormalization()(x)
    
    # SE block
    squeeze = layers.GlobalAveragePooling2D()(x)
    excitation = layers.Dense(64//16, activation='relu',
                            kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(squeeze)
    excitation = layers.Dropout(DROPOUT_RATE)(excitation)
    excitation = layers.Dense(64, activation='sigmoid')(excitation)
    excitation = layers.Reshape((1,1,64))(excitation)
    scale = layers.Multiply()([x, excitation])
    
    # Additional layers for better feature extraction
    scale = layers.Conv2D(128, (3,3), padding='same', activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(scale)
    scale = layers.BatchNormalization()(scale)
    scale = layers.MaxPooling2D((2,2))(scale)
    scale = layers.Dropout(DROPOUT_RATE)(scale)
    
    return Model(i, scale)

def build_capsulenet(s):
    i = Input(s)
    x = layers.Conv2D(256, (9,9), activation='relu', padding='same')(i)
    x = layers.Conv2D(256, (9,9), strides=2, activation='relu', padding='same')(x)
    return Model(i, x)

def build_hrnet(s):
    i = Input(s)
    h1 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(i)
    l1 = layers.Conv2D(64, (3,3), strides=2, padding='same', activation='relu')(i)
    l1_up = layers.UpSampling2D()(l1)
    out = layers.Add()([h1, l1_up])
    return Model(i, out)

def build_vgg16(s): return VGG16(include_top=False, weights='imagenet', input_shape=s)
def build_vgg19(s): return VGG19(include_top=False, weights='imagenet', input_shape=s)
def build_inception_v3(s): return InceptionV3(include_top=False, weights='imagenet', input_shape=s)
def build_resnet50(s): return ResNet50(include_top=False, weights='imagenet', input_shape=s)
def build_resnet101(s): return ResNet101(include_top=False, weights='imagenet', input_shape=s)
def build_resnet152(s): return ResNet152(include_top=False, weights='imagenet', input_shape=s)
def build_inception_resnet_v2(s): return InceptionResNetV2(include_top=False, weights='imagenet', input_shape=s)
def build_xception(s): return Xception(include_top=False, weights='imagenet', input_shape=s)
def build_densenet121(s): return DenseNet121(include_top=False, weights='imagenet', input_shape=s)
def build_densenet201(s): return DenseNet201(include_top=False, weights='imagenet', input_shape=s)
def build_mobilenet_v2(s): return MobileNetV2(include_top=False, weights='imagenet', input_shape=s)

MODEL_FACTORY = {
    'AlexNet': {'builder': build_alexnet, 'preprocess': lambda x: x/255.0},
    'NIN': {'builder': build_nin, 'preprocess': lambda x: x/255.0},
    'ZFNet': {'builder': build_zfnet, 'preprocess': lambda x: x/255.0},
    'VGG16': {'builder': build_vgg16, 'preprocess': vgg16.preprocess_input},
    'VGG19': {'builder': build_vgg19, 'preprocess': vgg19.preprocess_input},
    'GoogLeNet': {'builder': build_googlenet, 'preprocess': lambda x: x/255.0},
    'InceptionV3': {'builder': build_inception_v3, 'preprocess': inception_v3.preprocess_input},
    'Highway': {'builder': build_highway, 'preprocess': lambda x: x/255.0},
    'InceptionV4': {'builder': build_inception_v4, 'preprocess': lambda x: x/255.0},
    'ResNet50': {'builder': build_resnet50, 'preprocess': resnet.preprocess_input},
    'ResNet101': {'builder': build_resnet101, 'preprocess': resnet.preprocess_input},
    'ResNet152': {'builder': build_resnet152, 'preprocess': resnet.preprocess_input},
    'InceptionResNetV2': {'builder': build_inception_resnet_v2, 'preprocess': inception_resnet_v2.preprocess_input},
    'FractalNet': {'builder': build_fractalnet, 'preprocess': lambda x: x/255.0},
    'WideResNet': {'builder': build_wideresnet, 'preprocess': lambda x: x/255.0},
    'Xception': {'builder': build_xception, 'preprocess': xception.preprocess_input},
    'ResidualAttention': {'builder': build_residual_attention, 'preprocess': lambda x: x/255.0},
    'SENet': {'builder': build_senet, 'preprocess': lambda x: x/255.0},
    'DenseNet121': {'builder': build_densenet121, 'preprocess': densenet.preprocess_input},
    'DenseNet201': {'builder': build_densenet201, 'preprocess': densenet.preprocess_input},
    'MobileNetV2': {'builder': build_mobilenet_v2, 'preprocess': mobilenet_v2.preprocess_input},
    'CapsuleNet': {'builder': build_capsulenet, 'preprocess': lambda x: x/255.0},
    'HRNet': {'builder': build_hrnet, 'preprocess': lambda x: x/255.0}
}

print(f"✓ Defined {len(MODEL_FACTORY)} model architectures")

✓ Defined 23 model architectures


## 3. Multi-Modal Training Orchestration

In [7]:
# Refined model registry with freezing/fine-tune metadata
MODEL_FACTORY = {
    'AlexNet': {'builder': build_alexnet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'NIN': {'builder': build_nin, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'ZFNet': {'builder': build_zfnet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'VGG16': {'builder': build_vgg16, 'preprocess': vgg16.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.4},
    'VGG19': {'builder': build_vgg19, 'preprocess': vgg19.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.4},
    'GoogLeNet': {'builder': build_googlenet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'InceptionV3': {'builder': build_inception_v3, 'preprocess': inception_v3.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.4},
    'Highway': {'builder': build_highway, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'InceptionV4': {'builder': build_inception_v4, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'ResNet50': {'builder': build_resnet50, 'preprocess': resnet.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.3},
    'ResNet101': {'builder': build_resnet101, 'preprocess': resnet.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.3},
    'ResNet152': {'builder': build_resnet152, 'preprocess': resnet.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.3},
    'InceptionResNetV2': {'builder': build_inception_resnet_v2, 'preprocess': inception_resnet_v2.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.4},
    'FractalNet': {'builder': build_fractalnet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'WideResNet': {'builder': build_wideresnet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'Xception': {'builder': build_xception, 'preprocess': xception.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.4},
    'ResidualAttention': {'builder': build_residual_attention, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'SENet': {'builder': build_senet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'DenseNet121': {'builder': build_densenet121, 'preprocess': densenet.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.35},
    'DenseNet201': {'builder': build_densenet201, 'preprocess': densenet.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.35},
    'MobileNetV2': {'builder': build_mobilenet_v2, 'preprocess': mobilenet_v2.preprocess_input, 'freeze_backbone': True, 'fine_tune_fraction': 0.5},
    'CapsuleNet': {'builder': build_capsulenet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False},
    'HRNet': {'builder': build_hrnet, 'preprocess': lambda x: x/255.0, 'freeze_backbone': False}
}

print(f"[Setup] MODEL_FACTORY refreshed with {len(MODEL_FACTORY)} entries")


[Setup] MODEL_FACTORY refreshed with 23 entries


In [8]:
if 'multimodal_results' in globals() and len(df_with_images) > 0 and multimodal_results:
    ranked = sorted(multimodal_results.items(), key=lambda kv: kv[1])
    best_name, best_mae = ranked[0]
    print("\nMulti-modal leaderboard (lower MAE is better):")
    for pos, (name, mae) in enumerate(ranked, 1):
        print(f"  {pos:>2}. {name:<20} MAE={mae:.4f}")
    print(f"\n→ Best multi-modal model: {best_name} (val MAE={best_mae:.4f})")
    import json
    leaderboard_path = SAVE_DIR / 'multimodal_leaderboard.json'
    def _write_leaderboard(path=leaderboard_path):
        with open(path, 'w') as f:
            json.dump(ranked, f, indent=2)
    safe_artifact_write("Leaderboard", leaderboard_path, _write_leaderboard)
elif len(df_with_images) > 0:
    print("\nNo multi-modal models finished training successfully yet. Run the training cell first.")
else:
    print("\nNo properties with images available.")




No multi-modal models finished training successfully yet. Run the training cell first.


In [9]:
def create_enhanced_multimodal_model(backbone, tab_shape, dropout_rate=DROPOUT_RATE, l2_reg=L2_REG):
    """Enhanced multimodal model with attention mechanism and better fusion - Tuned for MAE <= 0.5"""
    
    # Enhanced image branch with attention - Increased capacity
    x_img = layers.GlobalAveragePooling2D()(backbone.output)
    x_img_max = layers.GlobalMaxPooling2D()(backbone.output)
    x_img = layers.concatenate([x_img, x_img_max])  # Combine avg and max pooling
    
    x_img = layers.BatchNormalization()(x_img)
    x_img = layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_img)  # Increased from 512
    x_img = layers.LeakyReLU(0.1)(x_img)
    x_img = layers.Dropout(dropout_rate)(x_img)
    
    x_img = layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_img)
    x_img = layers.BatchNormalization()(x_img)
    x_img = layers.LeakyReLU(0.1)(x_img)
    x_img = layers.Dropout(dropout_rate * 0.8)(x_img)
    
    x_img = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_img)  # Additional layer
    x_img = layers.BatchNormalization()(x_img)
    x_img = layers.LeakyReLU(0.1)(x_img)
    x_img = layers.Dropout(dropout_rate * 0.6)(x_img)
    
    # Enhanced tabular branch - Increased capacity
    input_tab = Input(tab_shape, name='tabular')
    x_tab = layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(input_tab)  # Increased from 512
    x_tab = layers.BatchNormalization()(x_tab)
    x_tab = layers.LeakyReLU(0.1)(x_tab)
    x_tab = layers.Dropout(dropout_rate * 0.8)(x_tab)
    
    x_tab = layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_tab)
    x_tab = layers.BatchNormalization()(x_tab)
    x_tab = layers.LeakyReLU(0.1)(x_tab)
    x_tab = layers.Dropout(dropout_rate * 0.7)(x_tab)
    
    x_tab = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_tab)
    x_tab = layers.BatchNormalization()(x_tab)
    x_tab = layers.LeakyReLU(0.1)(x_tab)
    x_tab = layers.Dropout(dropout_rate * 0.6)(x_tab)
    
    x_tab = layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x_tab)  # Additional layer
    x_tab = layers.BatchNormalization()(x_tab)
    x_tab = layers.LeakyReLU(0.1)(x_tab)
    x_tab = layers.Dropout(dropout_rate * 0.4)(x_tab)
    
    # Cross-attention mechanism
    img_attention = layers.Dense(256, activation='softmax')(x_img)
    tab_attention = layers.Dense(128, activation='softmax')(x_tab)
    
    x_img_attended = layers.Multiply()([x_img, img_attention])
    x_tab_attended = layers.Multiply()([x_tab, tab_attention])
    
    # Enhanced fusion - Increased capacity
    x = layers.concatenate([x_img_attended, x_tab_attended])
    
    # Residual connections in fusion layers
    x1 = layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)  # Increased from 384
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.LeakyReLU(0.1)(x1)
    x1 = layers.Dropout(dropout_rate)(x1)
    
    x2 = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.LeakyReLU(0.1)(x2)
    x2 = layers.Dropout(dropout_rate * 0.8)(x2)
    
    x3 = layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.LeakyReLU(0.1)(x3)
    x3 = layers.Dropout(dropout_rate * 0.6)(x3)
    
    x4 = layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x3)  # Additional layer
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.LeakyReLU(0.1)(x4)
    x4 = layers.Dropout(dropout_rate * 0.4)(x4)
    
    # Final prediction with multiple heads for ensemble-like behavior
    out1 = layers.Dense(32, activation='relu')(x4)
    out1 = layers.Dense(1, name='main_output')(out1)
    
    # Auxiliary output for regularization
    aux_out = layers.Dense(16, activation='relu')(x3)
    aux_out = layers.Dense(1, name='aux_output')(aux_out)
    
    model = Model([backbone.input, input_tab], [out1, aux_out])
    
    return model

multimodal_results = {}
model_performance = []
failed = []

if len(df_with_images) > 0:
    indices = df_with_images.index
    X_tab_mm = X_all_tabular[indices]
    y_mm = df_with_images['log_price_std'].values
    
    print("\n[Stage] Preparing multimodal tensors...")
    raw_imgs = np.array([load_raw_image(p) for p in df_with_images['image_path']])
    
    stratify_labels = price_bins_with_images.values if price_bins_with_images is not None else None
    X_tab_train, X_tab_val, X_img_raw_train, X_img_raw_val, y_train, y_val = train_test_split(
        X_tab_mm,
        raw_imgs,
        y_mm,
        test_size=0.2,
        random_state=42,
        stratify=stratify_labels
    )
    
    print(f"[Split] train={len(X_tab_train)} | val={len(X_tab_val)}")
    print("\n--- Multimodal docket ---")
    print("Multi-modal roster (image + tabular) in flight")
    print("-----------------------------\n")
    
    # Prepare training data format (needed for hyperparameter search)
    y_train_multi = [y_train, y_train]  # Main and auxiliary outputs
    y_val_multi = [y_val, y_val]
    
    for idx, (name, cfg) in enumerate(MODEL_FACTORY.items(), 1):
        print(f"[Model {idx}/{len(MODEL_FACTORY)}] {name}")
        try:
            X_img_train = cfg['preprocess'](X_img_raw_train.copy().astype(np.float32))
            X_img_val = cfg['preprocess'](X_img_raw_val.copy().astype(np.float32))
            
            tf.keras.backend.clear_session()
            backbone = cfg['builder'](IMG_SIZE + (3,))
            if cfg.get('freeze_backbone', False):
                freeze_entire_backbone(backbone)
            else:
                backbone.trainable = True
            
            # Use static hyperparameters to keep runs deterministic and avoid repeated searches
            optimal_lr = cfg.get('learning_rate', INITIAL_LR)
            optimal_dropout = cfg.get('dropout_rate', DROPOUT_RATE)
            optimal_batch = cfg.get('batch_size', BATCH_SIZE)
            
            # Create model with optimal hyperparameters
            model = create_enhanced_multimodal_model(backbone, (X_tab_mm.shape[1],), dropout_rate=optimal_dropout)

            # Parameter summary per architecture (Keras already tracks this)
            total_params = model.count_params()
            trainable_params = sum(int(tf.size(w)) for w in model.trainable_weights)
            non_trainable_params = total_params - trainable_params
            print(f"    [Params] total={total_params:,} | trainable={trainable_params:,} | non-trainable={non_trainable_params:,}")
            
            # Compile model with optimal learning rate
            compile_multimodal(model, optimal_lr)
            
            # Use modern Keras format (.keras) instead of legacy HDF5
            ckpt_path = str(SAVE_DIR / f'multimodal_{name}.keras')
            
            # Enhanced callbacks with advanced scheduling
            cbs = build_multimodal_callbacks(ckpt_path, patience=10, use_scheduler=True, lr_epochs=EPOCHS, initial_lr=optimal_lr)
            
            # Train with HDF5 warning suppression (y_train_multi and y_val_multi already defined above)
            with suppress_hdf5_warnings():
                hist = model.fit(
                    [X_img_train, X_tab_train], 
                    y_train_multi, 
                    validation_data=([X_img_val, X_tab_val], y_val_multi), 
                    epochs=EPOCHS, 
                    batch_size=optimal_batch,  # Use optimal batch size from hyperparameter search
                    callbacks=cbs, 
                    verbose=1,
                    shuffle=True
                )

            history = hist.history
            best_idx = int(np.argmin(history['val_main_output_mae']))
            best_epoch = best_idx + 1

            def _extract(metric_name):
                values = history.get(metric_name)
                return values[best_idx] if values is not None else np.nan

            best_val_loss = _extract('val_loss')
            best_val_mae = _extract('val_main_output_mae')
            best_val_mse = _extract('val_main_output_mse')
            best_aux_mae = _extract('val_aux_output_mae')

            # CRITICAL: Load the best model weights from checkpoint (not the last epoch)
            if ENABLE_MODEL_SAVING and Path(ckpt_path).exists():
                try:
                    best_model = tf.keras.models.load_model(ckpt_path)
                    model.set_weights(best_model.get_weights())
                    del best_model
                    print(f"  ✓ Loaded best weights from epoch {best_epoch} (val MAE: {best_val_mae:.4f})")
                except Exception as load_error:
                    print(f"  ⚠ Could not explicitly load best weights: {load_error}")
            else:
                print("  ↪ Skipping explicit best-weight reload (saving disabled or checkpoint missing). Using in-memory weights.")

            preds_log = model.predict([X_img_val, X_tab_val], verbose=0)[0].squeeze()
            train_preds_log = model.predict([X_img_train, X_tab_train], verbose=0)[0].squeeze()
            baseline_metrics, calibrated_metrics, calibration_params, calibrated_val_std = evaluate_with_calibration(
                train_preds_log,
                y_train,
                preds_log,
                y_val
            )

            print(f"    [Checkpoint] best epoch = {best_epoch}")
            print(f"    [Val] loss={best_val_loss:.4f} | main MAE={best_val_mae:.4f} | main MSE={best_val_mse:.4f} | aux MAE={best_aux_mae:.4f}")
            print(f"    [Baseline USD] MAE={baseline_metrics[0]:.4f} | MSE={baseline_metrics[1]:.4f} | R²={baseline_metrics[2]:.4f}")
            print(f"    [Calibrated USD] MAE={calibrated_metrics[0]:.4f} | MSE={calibrated_metrics[1]:.4f} | R²={calibrated_metrics[2]:.4f}")
            actual_mae, actual_mse, actual_r2 = calibrated_metrics

            fine_tuned = False
            if cfg.get('freeze_backbone', False) and best_val_mae > TARGET_VAL_MAE:
                fine_tuned = True
                print(f"    [Fine-tune] releasing top {int(cfg.get('fine_tune_fraction', 0.4) * 100)}% of layers (val MAE={best_val_mae:.3f})")
                unfreeze_backbone_fraction(backbone, cfg.get('fine_tune_fraction', 0.4))
                compile_multimodal(model, FINE_TUNE_LR)
                fine_cbs = build_multimodal_callbacks(ckpt_path, patience=5, use_scheduler=False, lr_epochs=FINE_TUNE_EPOCHS)
                with suppress_hdf5_warnings():
                    fine_hist = model.fit(
                        [X_img_train, X_tab_train], 
                        y_train_multi, 
                        validation_data=([X_img_val, X_tab_val], y_val_multi), 
                        epochs=FINE_TUNE_EPOCHS, 
                        batch_size=max(16, BATCH_SIZE // 2), 
                        callbacks=fine_cbs, 
                        verbose=1,
                        shuffle=True
                    )
                fine_history = fine_hist.history
                fine_idx = int(np.argmin(fine_history['val_main_output_mae']))
                fine_mae = fine_history['val_main_output_mae'][fine_idx]
                
                # CRITICAL: Load the best fine-tuned model weights (not the last epoch)
                if ENABLE_MODEL_SAVING and Path(ckpt_path).exists():
                    try:
                        best_fine_model = tf.keras.models.load_model(ckpt_path)
                        model.set_weights(best_fine_model.get_weights())
                        del best_fine_model
                        print(f"  ✓ Loaded best fine-tuned weights from epoch {fine_idx + 1} (val MAE: {fine_mae:.4f})")
                    except Exception as load_error:
                        print(f"  ⚠ Could not explicitly load best fine-tuned weights: {load_error}")
                else:
                    print("  ↪ Skipping explicit fine-tuned weight reload (saving disabled or checkpoint missing). Using in-memory weights.")
                
                if fine_mae <= best_val_mae:
                    best_val_mae = fine_mae
                    if 'val_loss' in fine_history:
                        best_val_loss = fine_history['val_loss'][fine_idx]
                    if 'val_main_output_mse' in fine_history:
                        best_val_mse = fine_history['val_main_output_mse'][fine_idx]
                    if 'val_aux_output_mae' in fine_history:
                        best_aux_mae = fine_history['val_aux_output_mae'][fine_idx]
                    best_epoch = f"FT-{fine_idx + 1}"
                preds_log = model.predict([X_img_val, X_tab_val], verbose=0)[0].squeeze()
                actual_mae, actual_mse, actual_r2 = evaluate_actual_metrics(preds_log, y_val)
                print(f"    [Fine-tune USD] MAE: {actual_mae:.4f} | MSE: {actual_mse:.4f} | R²: {actual_r2:.4f}")

            multimodal_results[name] = best_val_mae
            model_performance.append({
                'model': name,
                'type': 'multimodal',
                'best_epoch': best_epoch,
                'val_loss': float(best_val_loss),
                'val_main_mae': float(best_val_mae),
                'val_main_mse': float(best_val_mse),
                'val_aux_mae': float(best_aux_mae),
                'actual_mae': float(actual_mae),
                'actual_mse': float(actual_mse),
                'actual_r2': float(actual_r2),
                'baseline_actual_mae': float(baseline_metrics[0]),
                'baseline_actual_r2': float(baseline_metrics[2]),
                'calibration': calibration_params,
                'fine_tuned': fine_tuned
            })

            model_key = f"multimodal::{name}"
            if ENABLE_MODEL_SAVING:
                if should_update_best_model(model_key, actual_r2):
                    def _save_multimodal(path=ckpt_path):
                        with suppress_hdf5_warnings():
                            model.save(path, overwrite=True)
                    saved = safe_artifact_write(f"{name} multimodal weights", ckpt_path, _save_multimodal)
                    if saved:
                        BEST_MODEL_LEDGER[model_key] = {
                            'r2': float(actual_r2),
                            'mae': float(actual_mae),
                            'baseline_r2': float(baseline_metrics[2]),
                            'baseline_mae': float(baseline_metrics[0]),
                            'timestamp': datetime.utcnow().isoformat()
                        }
                        persist_best_model_ledger()
                        print(f"    [R^2 ledger] Recorded new best for {model_key}")
                else:
                    previous = BEST_MODEL_LEDGER.get(model_key)
                    if previous:
                        print(f"    [R^2 ledger] Existing best R^2 ({previous['r2']:.4f}) >= current ({actual_r2:.4f}); checkpoint unchanged.")
            else:
                print("    [R^2 ledger] Saving disabled; best-check skip triggered.")

            del model, backbone
            tf.keras.backend.clear_session()
            gc.collect()
        except Exception as e:
            failed.append(name)
            print(f"  ✗ FAILED: {e}")
            tf.keras.backend.clear_session()
            gc.collect()

    print(f"\n[Result] Multimodal stage finished: {len(multimodal_results)}/{len(MODEL_FACTORY)} succeeded")
else:
    print("\n⚠ No properties with images found. Skipping multi-modal training.")


[Stage] Preparing multimodal tensors...
[Split] train=204 | val=52

--- Multimodal docket ---
Multi-modal roster (image + tabular) in flight
-----------------------------

[Model 1/23] AlexNet
    [Params] total=6,148,130 | trainable=6,137,058 | non-trainable=11,072
Epoch 1/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - aux_output_loss: 0.9083 - aux_output_mae: 1.3514 - loss: 1.3437 - main_output_loss: 0.5528 - main_output_mae: 0.9552 - main_output_mse: 1.3952
Epoch 1: val_main_output_mae improved from None to 0.71472, saving model to D:\houses_saved/multimodal_AlexNet.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 423ms/step - aux_output_loss: 0.8802 - aux_output_mae: 1.2941 - loss: 1.3465 - main_output_loss: 0.5592 - main_output_mae: 0.9700 - main_output_mse: 1.4801 - val_aux_output_loss: 0.3492 - val_aux_output_mae: 0.7179 - val_loss: 0.9820 - val_main_output_loss: 0.3475 - val_main_output_mae: 0.7147 - val_main_output_mse: 0.7765 - learning_rate: 5.0000e-05
Epoch 2/60
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s

## 4. Purpose-Built Tabular Network

In [4]:
def create_enhanced_tabular_model(input_shape):
    """Enhanced tabular model with residual connections and attention"""
    i = Input(input_shape, name='tabular_input')
    
    # Input normalization
    x = layers.BatchNormalization()(i)
    
    # First block with residual connection
    x1 = layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.LeakyReLU(0.1)(x1)
    x1 = layers.Dropout(DROPOUT_RATE)(x1)
    
    x2 = layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x1)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.LeakyReLU(0.1)(x2)
    x2 = layers.Dropout(DROPOUT_RATE * 0.8)(x2)
    
    # Residual connection
    x2_res = layers.Add()([x1, x2])
    
    # Second block with attention
    x3 = layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x2_res)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.LeakyReLU(0.1)(x3)
    x3 = layers.Dropout(DROPOUT_RATE * 0.7)(x3)
    
    # Self-attention mechanism
    attention_weights = layers.Dense(256, activation='softmax')(x3)
    x3_attended = layers.Multiply()([x3, attention_weights])
    
    # Third block
    x4 = layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x3_attended)
    x4 = layers.BatchNormalization()(x4)
    x4 = layers.LeakyReLU(0.1)(x4)
    x4 = layers.Dropout(DROPOUT_RATE * 0.6)(x4)
    
    # Fourth block
    x5 = layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(L2_REG))(x4)
    x5 = layers.BatchNormalization()(x5)
    x5 = layers.LeakyReLU(0.1)(x5)
    x5 = layers.Dropout(DROPOUT_RATE * 0.4)(x5)
    
    # Multiple prediction heads for ensemble-like behavior
    out1 = layers.Dense(32, activation='relu')(x5)
    out1 = layers.Dense(1, name='main_output')(out1)
    
    # Auxiliary output from earlier layer
    aux_out = layers.Dense(16, activation='relu')(x4)
    aux_out = layers.Dense(1, name='aux_output')(aux_out)
    
    model = Model(i, [out1, aux_out])
    
    # Enhanced optimizer and loss
    model.compile(
        optimizer=optimizers.AdamW(learning_rate=INITIAL_LR, weight_decay=L2_REG, clipnorm=1.0),
        loss={
            'main_output': tf.keras.losses.Huber(delta=1.0),
            'aux_output': tf.keras.losses.Huber(delta=1.0)
        },
        loss_weights={'main_output': 1.0, 'aux_output': 0.2},
        metrics={
            'main_output': ['mae', 'mse'],
            'aux_output': ['mae']
        }
    )
    return model

tabular_mae = None
tabular_actual_metrics = None

if len(df_no_images) > 0:
    print("\n--- Tabular-only stage ---")
    print("Tabular specialist training (no images)")
    print("-----------------------------\n")
    
    indices_no_img = df_no_images.index
    X_tab_no_img = X_all_tabular[indices_no_img]
    y_no_img = df_no_images['log_price_std'].values
    
    stratify_no_img = price_bins_no_images.values if price_bins_no_images is not None else None
    X_train, X_val, y_train, y_val = train_test_split(
        X_tab_no_img,
        y_no_img,
        test_size=0.2,
        random_state=42,
        stratify=stratify_no_img
    )
    print(f"[Split] train={len(X_train)} | val={len(X_val)}")
    
    try:
        tab_model = create_enhanced_tabular_model((X_tab_no_img.shape[1],))
        
        # Use modern Keras format
        ckpt_path = str(SAVE_DIR / 'tabular_only_model.keras')
        
        # Enhanced callbacks
        cbs = []
        if ENABLE_MODEL_SAVING:
            cbs.append(
                KerasModelCheckpoint(
                    ckpt_path, 
                    save_best_only=True, 
                    monitor='val_main_output_mae',
                    mode='min',
                    verbose=1
                )
            )
        else:
            cbs.append(SkipCheckpointNotice(ckpt_path))
        cbs.extend([
            callbacks.ReduceLROnPlateau(
                monitor='val_main_output_mae', 
                factor=0.7, 
                patience=8, 
                min_lr=MIN_LR, 
                mode='min',
                verbose=1,
                cooldown=3
            ),
            callbacks.LearningRateScheduler(
                lambda epoch: INITIAL_LR * 0.5 * (1 + np.cos(np.pi * epoch / EPOCHS)),
                verbose=0
            ),
            GeneralizationGuard(
                monitor='val_main_output_mae',
                reference='main_output_mae',
                gap=0.03,
                patience=4,
                start_epoch=12,
                min_lr=MIN_LR,
                decay=0.65
            ),
            callbacks.TerminateOnNaN()
        ])
        
        # Enhanced training with auxiliary loss
        y_train_multi = [y_train, y_train]
        y_val_multi = [y_val, y_val]
        
        # Train with HDF5 warning suppression
        with suppress_hdf5_warnings():
            hist = tab_model.fit(
                X_train, 
                y_train_multi, 
                validation_data=(X_val, y_val_multi), 
                epochs=EPOCHS, 
                batch_size=64,  # Larger batch size for tabular data
                callbacks=cbs, 
                verbose=1,
                shuffle=True
            )
        
        # Find the best epoch based on validation MAE (not the last epoch)
        best_tab_idx = int(np.argmin(hist.history['val_main_output_mae']))
        best_tab_epoch = best_tab_idx + 1
        tabular_mae = hist.history['val_main_output_mae'][best_tab_idx]
        
        # CRITICAL: Load the best model weights from checkpoint (not the last epoch)
        if ENABLE_MODEL_SAVING and Path(ckpt_path).exists():
            try:
                best_tab_model = tf.keras.models.load_model(ckpt_path)
                tab_model.set_weights(best_tab_model.get_weights())
                del best_tab_model
                print(f"✓ Loaded best tabular model weights from epoch {best_tab_epoch} (val MAE: {tabular_mae:.4f})")
            except Exception as load_error:
                print(f"⚠ Could not explicitly load best tabular weights, continuing with in-memory weights: {load_error}")
        else:
            print("↪ Skipping explicit tabular weight reload (saving disabled or checkpoint missing). Using in-memory weights.")
        
        tab_preds_log = tab_model.predict(X_val, verbose=0)[0].squeeze()
        train_tab_preds_log = tab_model.predict(X_train, verbose=0)[0].squeeze()
        baseline_tab_metrics, calibrated_tab_metrics, tab_calibration_params, calibrated_tab_vals = evaluate_with_calibration(
            train_tab_preds_log,
            y_train,
            tab_preds_log,
            y_val
        )
        tabular_actual_metrics = {
            'baseline_mae': float(baseline_tab_metrics[0]),
            'baseline_r2': float(baseline_tab_metrics[2]),
            'mae': float(calibrated_tab_metrics[0]),
            'mse': float(calibrated_tab_metrics[1]),
            'r2': float(calibrated_tab_metrics[2]),
            'calibration': tab_calibration_params
        }
        print(f"✓ Tabular baseline actual metrics → MAE: {baseline_tab_metrics[0]:.4f} | MSE: {baseline_tab_metrics[1]:.4f} | R²: {baseline_tab_metrics[2]:.4f}")
        print(f"✓ Tabular calibrated actual metrics → MAE: {calibrated_tab_metrics[0]:.4f} | MSE: {calibrated_tab_metrics[1]:.4f} | R²: {calibrated_tab_metrics[2]:.4f}")
        
        print(f"\n[Result] Tabular validation MAE: {tabular_mae:.4f}")
        tabular_key = "tabular::base"
        if ENABLE_MODEL_SAVING:
            if should_update_best_model(tabular_key, tabular_actual_metrics['r2']):
                def _save_tabular(path=ckpt_path):
                    with suppress_hdf5_warnings():
                        tab_model.save(path, overwrite=True)
                saved = safe_artifact_write("Tabular model weights", ckpt_path, _save_tabular)
                if saved:
                    BEST_MODEL_LEDGER[tabular_key] = {
                        'r2': float(tabular_actual_metrics['r2']),
                        'mae': float(tabular_actual_metrics['mae']),
                        'baseline_r2': float(tabular_actual_metrics['baseline_r2']),
                        'baseline_mae': float(tabular_actual_metrics['baseline_mae']),
                        'timestamp': datetime.utcnow().isoformat()
                    }
                    persist_best_model_ledger()
                    print("[R^2 ledger] Updated tabular entry.")
            else:
                previous = BEST_MODEL_LEDGER.get(tabular_key)
                if previous:
                    print(f"[R^2 ledger] Existing tabular R?? ({previous['r2']:.4f}) >= {tabular_actual_metrics['r2']:.4f}; checkpoint unchanged.")
        else:
            print("[R^2 ledger] Saving disabled; best-check skip triggered for tabular model.")
        
        # Save training history for analysis
        import json
        history_path = SAVE_DIR / 'tabular_training_history.json'
        def _write_history(path=history_path):
            with open(path, 'w') as f:
                json.dump(hist.history, f, indent=2)
        safe_artifact_write("Tabular training history", history_path, _write_history)
        
    except Exception as e:
        print(f"✗ Enhanced tabular model training failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n⚠ No properties without images. Skipping tabular-only model.")

NameError: name 'df_no_images' is not defined

## 5. Blending Stack and Ranking

In [5]:
def create_ensemble_predictions():
    """Create ensemble predictions from top performing models"""
    if not multimodal_results:
        print("⚠ No multimodal results available for ensemble")
        return None
    
    # Select top 3 models for ensemble
    top_models = sorted(multimodal_results.items(), key=lambda x: x[1])[:3]
    print(f"🏆 Top 3 models for ensemble:")
    for i, (name, mae) in enumerate(top_models, 1):
        print(f"  {i}. {name}: MAE {mae:.4f}")
    
    # Load top models and create ensemble
    ensemble_models = []
    model_weights = []
    
    for name, mae in top_models:
        try:
            model_path = SAVE_DIR / f'multimodal_{name}.keras'
            if model_path.exists():
                model = tf.keras.models.load_model(model_path)
                ensemble_models.append((name, model))
                # Weight inversely proportional to MAE (better models get higher weight)
                weight = 1.0 / (mae + 1e-6)
                model_weights.append(weight)
                print(f"  ✓ Loaded {name}")
        except Exception as e:
            print(f"  ✗ Failed to load {name}: {e}")
    
    if ensemble_models:
        # Normalize weights
        total_weight = sum(model_weights)
        model_weights = [w / total_weight for w in model_weights]
        
        print(f"\\n✓ Ensemble created with {len(ensemble_models)} models")
        print(\"Model weights:\")
        for (name, _), weight in zip(ensemble_models, model_weights):
            print(f\"  {name}: {weight:.3f}\")
        
        # Save ensemble configuration
        ensemble_config = {
            'models': [name for name, _ in ensemble_models],
            'weights': model_weights,
            'top_models_performance': dict(top_models)
        }
        
        import json
        config_path = SAVE_DIR / 'ensemble_config.json'
        def _write_ensemble_config(path=config_path):
            with open(path, 'w') as f:
                json.dump(ensemble_config, f, indent=2)
        safe_artifact_write("Ensemble configuration", config_path, _write_ensemble_config)
        
        return ensemble_models, model_weights
    
    return None

def save_model_performance_report():
    \"\"\"Save comprehensive performance report\"\"\"
    report = {
        'training_config': {
            'img_size': IMG_SIZE,
            'batch_size': BATCH_SIZE,
            'epochs': EPOCHS,
            'initial_lr': INITIAL_LR,
            'min_lr': MIN_LR,
            'dropout_rate': DROPOUT_RATE,
            'l2_reg': L2_REG,
            'use_augmentation': USE_AUGMENTATION,
            'use_mixup': USE_MIXUP,
            'mixup_alpha': MIXUP_ALPHA
        },
        'data_info': {
            'total_properties': len(df),
            'properties_with_images': len(df_with_images) if len(df_with_images) > 0 else 0,
            'properties_without_images': len(df_no_images) if len(df_no_images) > 0 else 0,
            'tabular_features': X_all_tabular.shape[1] if X_all_tabular is not None else 0
        },
        'multimodal_results': multimodal_results,
        'model_performance': model_performance,
        'tabular_mae': tabular_mae,
        'tabular_actual_metrics': tabular_actual_metrics,
        'target_val_mae': TARGET_VAL_MAE,
        'failed_models': failed
    }
    
    # Add best model info
    if multimodal_results:
        best_model = min(multimodal_results.items(), key=lambda x: x[1])
        report['best_multimodal'] = {
            'name': best_model[0],
            'mae': best_model[1]
        }
    
    # Save report
    import json
    report_path = SAVE_DIR / 'training_report.json'
    def _write_report(path=report_path):
        with open(path, 'w') as f:
            json.dump(report, f, indent=2)
    safe_artifact_write("Training performance report", report_path, _write_report)
    
    return report

# Create ensemble and save reports
ensemble_result = create_ensemble_predictions()
performance_report = save_model_performance_report()

print(\"\\n\" + \"=\"*60)
print(\"ENSEMBLE AND REPORTING COMPLETE\")
print(\"=\"*60)


SyntaxError: unexpected character after line continuation character (3509080147.py, line 36)

## 6. End-of-Run Briefing


In [6]:
print("\n" + "="*68)
print("SESSION RECAP")
print("="*68)

print(f"[Overview] multimodal models run: {len(multimodal_results)}/{len(MODEL_FACTORY)}")
if failed:
    print(f"[Overview] models skipped/failed: {', '.join(failed)}")

if multimodal_results:
    print("\n[Leaderboard] Multimodal MAE ranking:")
    for i, (name, mae) in enumerate(sorted(multimodal_results.items(), key=lambda x: x[1]), 1):
        badge = "TOP1" if i == 1 else "TOP2" if i == 2 else "TOP3" if i == 3 else "RUN"
        print(f"  {badge:<4} #{i:02d} {name:20s} | MAE {mae:.4f}")
    best = min(multimodal_results.items(), key=lambda x: x[1])
    print(f"\n[Champion] {best[0]} | val MAE {best[1]:.4f}")
    if model_performance:
        champion_perf = next((mp for mp in model_performance if mp['model'] == best[0]), None)
        if champion_perf:
            status = 'yes' if champion_perf.get('fine_tuned') else 'no'
            print(f"    Calibrated USD metrics -> MAE {champion_perf['actual_mae']:.4f}, R² {champion_perf['actual_r2']:.4f}, fine-tuned: {status}")
            if 'baseline_actual_mae' in champion_perf:
                print(f"    Baseline USD metrics -> MAE {champion_perf['baseline_actual_mae']:.4f}, R² {champion_perf['baseline_actual_r2']:.4f}")
            if champion_perf.get('calibration'):
                cal = champion_perf['calibration']
                print(f"    Calibration fit -> slope {cal['slope']:.4f}, intercept {cal['intercept']:.4f}")

if tabular_mae:
    print(f"\n[Tabular] Validation MAE {tabular_mae:.4f}")
    if tabular_actual_metrics:
        print(f"    Calibrated USD metrics -> MAE {tabular_actual_metrics['mae']:.4f}, R² {tabular_actual_metrics['r2']:.4f}")
        print(f"    Baseline USD metrics -> MAE {tabular_actual_metrics['baseline_mae']:.4f}, R² {tabular_actual_metrics['baseline_r2']:.4f}")
        if tabular_actual_metrics.get('calibration'):
            cal = tabular_actual_metrics['calibration']
            print(f"    Calibration fit -> slope {cal['slope']:.4f}, intercept {cal['intercept']:.4f}")

print("\n[Artifacts]")
if ENABLE_MODEL_SAVING:
    print(f"  directory: {SAVE_DIR}")
    print("  format: .keras checkpoints + JSON reports")
else:
    print(f"  saving disabled; existing artifacts preserved at {SAVE_DIR}")

if ensemble_result:
    print("[Ensemble] blended predictions are ready for inference")

print("\n[Deployment notes]")
print("  - Use the multimodal champion or ensemble for listings with imagery.")
print("  - Route rows without pictures through the tabular expert.")
print("  - Load preprocessors from disk before inference to keep feature mapping consistent.")

print("\nSession closed.")



SESSION RECAP


NameError: name 'multimodal_results' is not defined

In [7]:

print("
" + "="*62)
print("RUN COMPLETE")
print("="*62 + "
")

print(f"[Count] multimodal models executed: {len(multimodal_results)}/{len(MODEL_FACTORY)}")
if failed:
    print(f"[Count] models skipped/failed: {', '.join(failed)}")

if multimodal_results:
    print("
[Multimodal ranking]")
    for name, mae in sorted(multimodal_results.items(), key=lambda x: x[1]):
        print(f"  {name:25s} | MAE {mae:.4f}")
    best = min(multimodal_results.items(), key=lambda x: x[1])
    print(f"
[Multimodal pick] {best[0]} | MAE {best[1]:.4f}")
    if model_performance:
        champion_perf = next((mp for mp in model_performance if mp["model"] == best[0]), None)
        if champion_perf:
            ft_flag = 'yes' if champion_perf.get("fine_tuned") else 'no'
            print(f"    Calibrated USD metrics -> MAE {champion_perf['actual_mae']:.4f}, R?? {champion_perf['actual_r2']:.4f}, fine-tuned: {ft_flag}")
            if 'baseline_actual_mae' in champion_perf:
                print(f"    Baseline USD metrics -> MAE {champion_perf['baseline_actual_mae']:.4f}, R?? {champion_perf['baseline_actual_r2']:.4f}")
            if champion_perf.get('calibration'):
                cal = champion_perf['calibration']
                print(f"    Calibration fit -> slope {cal['slope']:.4f}, intercept {cal['intercept']:.4f}")

if tabular_mae:
    print(f"
[Tabular model] MAE {tabular_mae:.4f}")
    if tabular_actual_metrics:
        print(f"    Calibrated USD metrics -> MAE {tabular_actual_metrics['mae']:.4f}, R?? {tabular_actual_metrics['r2']:.4f}")
        print(f"    Baseline USD metrics -> MAE {tabular_actual_metrics['baseline_mae']:.4f}, R?? {tabular_actual_metrics['baseline_r2']:.4f}")

if ENABLE_MODEL_SAVING:
    print(f"
[Artifacts] Stored at {SAVE_DIR}")
else:
    print(f"
[Artifacts] Saving disabled; nothing new written to {SAVE_DIR}")

print("
[Usage]")
print("??? Listings with imagery -> deploy the best multimodal checkpoint or the ensemble")
print("??? Listings without imagery -> rely on the tabular network")


SyntaxError: unterminated string literal (detected at line 1) (2246927500.py, line 1)